In [1]:
# 데이터 불러오기
import pandas as pd

DATA_PATH = 'C:/Work/1stDL/Roadbook_DeepLearning-master/TF_2.5v/code/clothes_classification/csv_data/nocolorinfo'

train_df = pd.read_csv(DATA_PATH + '/train.csv')
val_df = pd.read_csv(DATA_PATH + '/val.csv')
test_df = pd.read_csv(DATA_PATH + '/test.csv')

train_df.head()

,image,black,blue,brown,green,red,white,dress,shirt,pants,shorts,shoes
0,./clothes_dataset\blue_shorts\256d854b55ac32ea...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,./clothes_dataset\red_pants\584f778aece14f07c2...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,./clothes_dataset\green_pants\ec543ca241cefb2b...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,./clothes_dataset\brown_shorts\c8db9e0f7010592...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,./clothes_dataset\white_dress\551373c80717c5b0...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [2]:
# 이미지 제네레이터 정의 및 모델 구성하기

from tensorflow.keras.preprocessing.image import ImageDataGenerator
# 이미지 제네레이터를 정의합니다.
train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0 :
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size
    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential()
# 입력 데이터의 형태를 꼭 명시해야 합니다.
model.add(Flatten(input_shape = (112, 112, 3)))     # (112, 112, 3) -> (112 * 112 * 3)
model.add(Dense(128, activation = 'relu'))          # 128개의 출력을 가지는 Dense층 
model.add(Dense(64, activation = 'relu'))           # 64개의 출력을 가지는 Dense층
model.add(Dense(11, activation = 'sigmoid'))        # 11개의 출력을 가지는 신경망

model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['acc'])


C:\Users\human\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [33]:
# 데이터 제네레이터 정의하기
batch_size = 32
class_col = ['black', 'blue', 'brown', 'green', 'red', 'white', 'dress', 'shirt', 'pants', 'shorts', 'shoes']

# Make Generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../../Roadbook_DeepLearning-master/TF_2.5v/code/clothes_classification',
    x_col = 'image',
    y_col = class_col,
    target_size = (112, 112),
    color_mode='rgb',
    class_mode='raw',
    batch_size=batch_size,
    seed=42)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='C:/Work/1stDL/Roadbook_DeepLearning-master/TF_2.5v/code/clothes_classification',
    x_col = 'image',
    y_col = class_col,
    target_size = (112, 112),
    color_mode = 'rgb',
    class_mode='raw',
    batch_size=batch_size,
    shuffle=True)

Found 5578 validated image filenames.
Found 2391 validated image filenames.


In [34]:
# 제네레이터를 통해 모델 학습시키기
model.fit(train_generator, 
          steps_per_epoch=get_steps(len(train_df), batch_size), validation_data = val_generator, 
          validation_steps=get_steps(len(val_df), batch_size), epochs=10)

C:\Users\human\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 31s 178ms/step - acc: 0.5474 - loss: 0.2125 - val_acc: 0.5968 - val_loss: 0.2362
Epoch 2/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 177us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
  1/175 ━━━━━━━━━━━━━━━━━━━━ 28s 163ms/step - acc: 0.6875 - loss: 0.1863

c:\Dev\Anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


175/175 ━━━━━━━━━━━━━━━━━━━━ 32s 184ms/step - acc: 0.5517 - loss: 0.1962 - val_acc: 0.5784 - val_loss: 0.2103
Epoch 4/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 171us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 30s 173ms/step - acc: 0.5852 - loss: 0.1776 - val_acc: 0.5086 - val_loss: 0.2084
Epoch 6/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 183us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 31s 178ms/step - acc: 0.5801 - loss: 0.1735 - val_acc: 0.6763 - val_loss: 0.2032
Epoch 8/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 171us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 31s 177ms/step - acc: 0.5759 - loss: 0.1754 - val_acc: 0.6228 - val_loss: 0.2454
Epoch 10/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 201us/step - acc: 0.0000e+00 - loss: 0.0000e+00


In [35]:
test_datagen = ImageDataGenerator(rescale = 1./255)

# y_col: None, class_mode: None이므로
# test_generator는 image만 반환하고, label은 반환하지 않습니다.
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df, 
    directory=None,
    x_col = 'image',
    y_col = None,
    target_size = (112, 112),
    color_mode='rgb',
    class_mode=None,
    batch_size=batch_size,
    shuffle = False
)

Found 0 validated image filenames.


C:\Users\human\AppData\Roaming\Python\Python312\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 3416 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


In [36]:
# 해당 셀의 소스는 책에 있는게 계속 에러나서 강사님께서 직접 짜주심

def reshape_generator(generator):
    for batch in generator:
        yield (batch,)  # 튜플로 감싸서 반환



try:
    reshaped_generator = reshape_generator(test_generator)
    preds = model.predict(
        reshaped_generator,
        steps=len(test_df),
        verbose=1
    )
    print("예측 형태:", preds.shape)
except Exception as e:
    print("오류 발생:", str(e))
    
    # 추가 디버깅 정보
    batch = next(iter(test_generator))
    print("배치 형태:", batch.shape)
    print("배치 데이터 타입:", batch.dtype)

3416/3416 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
예측 형태: (0, 11)


In [37]:
# 테스트 데이터 예측하기
import matplotlib.pyplot as plt
import cv2

do_preds = preds[:8]

for i, pred in enumerate(do_preds):
    plt.subplot(2, 4, i+1)
    prob = zip(class_col, list(pred))
    prob = sorted(list(prob), key=lambda z: z[1], reverse = True)[:2]

    image = cv2.imread(test_df['image'][i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.imshow(image)
    plt.title(f'{prob[0][0]}: {round(prob[0][1] * 100, 2)}% /n {prob[1][0]}: {round(prob[1][1] * 100, 2)}%')
    
plt.tight_layout()

<Figure size 640x480 with 0 Axes>